In [2]:
import enum
from typing import List, Dict

class TipoProducto(enum.Enum):
    P10 = "P10"
    P16 = "P16"
    P20 = "P20"
    P30 = "P30"
    P40 = "P40"

class Tolva:
    def __init__(self, material: str, capacidad: float, altura_max: float):
        self.material = material
        self.capacidad = capacidad
        self.altura_max = altura_max
        self.nivel_actual = 0

    def tiempo_vaciado(self, consumo_por_hora: float) -> float:
        toneladas_reales = (self.nivel_actual * self.capacidad) / self.altura_max
        return toneladas_reales / consumo_por_hora if consumo_por_hora > 0 else float('inf')

    def alimentar(self, cantidad: float):
        self.nivel_actual = min(self.nivel_actual + cantidad, self.altura_max)

    def espacio_disponible(self) -> float:
        return self.capacidad - (self.nivel_actual * self.capacidad / self.altura_max)

class Molino:
    def __init__(self, nombre: str, tolvas: Dict[str, Tolva], rendimiento: float):
        self.nombre = nombre
        self.tolvas = tolvas
        self.rendimiento = rendimiento
        self.tipo_producto = None
        self.alimentacion_fresca = 0
        self.ratios = {}

    def set_producto(self, tipo_producto: TipoProducto, alimentacion_fresca: float, ratios: Dict[str, float]):
        self.tipo_producto = tipo_producto
        self.alimentacion_fresca = alimentacion_fresca
        self.ratios = ratios

    def tiempo_vaciado(self, material: str) -> float:
        if material not in self.ratios or material not in self.tolvas:
            return float('inf')
        consumo = (self.alimentacion_fresca * self.ratios[material]) / 100
        return self.tolvas[material].tiempo_vaciado(consumo)

class SistemaAlimentacion:
    def __init__(self):
        self.mc1 = Molino("MC1", {
            "clinker": Tolva("Clinker", 500, 14),
            "puzolana": Tolva("Puzolana", 300, 12),
            "yeso": Tolva("Yeso", 300, 10)
        }, 0.8)
        self.mc2 = Molino("MC2", {
            "clinker": Tolva("Clinker", 300, 9),
            "puzolana_humeda": Tolva("Puzolana Húmeda", 500, 15),
            "puzolana_seca": Tolva("Puzolana Seca", 100, 12),
            "yeso": Tolva("Yeso", 120, 9)
        }, 0.8)
        self.mc3 = Molino("MC3", {
            "clinker": Tolva("Clinker", 60, 100),
            "puzolana": Tolva("Puzolana", 35, 100),
            "yeso": Tolva("Yeso", 30, 100)
        }, 0.5)
        self.silo_blanco = Tolva("Clinker", 500, 10.5)
        self.silo_b = Tolva("Clinker", 500, 10)
        self.galpon = Tolva("Clinker", 1000, 20)
        self.alimentaciones_actuales = set()
        self.carga_banda_transporte = 0

    def set_productos(self):
        for molino in [self.mc1, self.mc2, self.mc3]:
            tipo_producto = self.solicitar_tipo_producto(molino.nombre)
            alimentacion_fresca, ratios = self.obtener_datos_producto(molino.nombre, tipo_producto)
            molino.set_producto(tipo_producto, alimentacion_fresca, ratios)

    def solicitar_tipo_producto(self, nombre_molino: str) -> TipoProducto:
        productos_disponibles = {
            "MC1": [TipoProducto.P30, TipoProducto.P40],
            "MC2": [TipoProducto.P10, TipoProducto.P16, TipoProducto.P20, TipoProducto.P30],
            "MC3": [TipoProducto.P30]
        }
        productos_str = ", ".join([tipo.value for tipo in productos_disponibles[nombre_molino]])
        print(f"\nTipos de producto disponibles para {nombre_molino}: {productos_str}")
        while True:
            seleccion = input(f"Seleccione el tipo de producto para {nombre_molino}: ").upper()
            try:
                tipo_producto = TipoProducto(seleccion)
                if tipo_producto in productos_disponibles[nombre_molino]:
                    return tipo_producto
                else:
                    print("Producto no disponible para este molino. Por favor, intente de nuevo.")
            except ValueError:
                print("Selección inválida. Por favor, intente de nuevo.")

    def obtener_datos_producto(self, nombre_molino: str, tipo_producto: TipoProducto) -> tuple:
        datos_productos = {
            "MC1": {
                TipoProducto.P30: (72, {"clinker": 68.5, "puzolana": 30, "yeso": 1.5}),
                TipoProducto.P40: (64, {"clinker": 58.5, "puzolana": 40, "yeso": 1.5})
            },
            "MC2": {
                TipoProducto.P10: (70, {"clinker": 87, "puzolana_humeda": 10, "yeso": 3}),
                TipoProducto.P16: (80, {"clinker": 81.5, "puzolana_humeda": 16, "yeso": 2.5}),
                TipoProducto.P20: (87, {"clinker": 85.5, "puzolana_humeda": 12, "yeso": 2.5}),
                TipoProducto.P30: (110, {"clinker": 68, "puzolana_humeda": 30, "yeso": 2})
            },
            "MC3": {
                TipoProducto.P30: (37, {"clinker": 67.5, "puzolana": 30, "yeso": 2.5})
            }
        }
        return datos_productos[nombre_molino][tipo_producto]

    def optimizar_alimentacion(self) -> List[str]:
        recomendaciones = []
        for molino in [self.mc1, self.mc2, self.mc3]:
            recomendaciones.extend(self.optimizar_molino(molino))
        return recomendaciones

    def optimizar_molino(self, molino: Molino) -> List[str]:
        print(f"\nOptimizando {molino.nombre}")
        print(f"Producto actual: {molino.tipo_producto.value}")
        print("Necesidades:")
        for material, ratio in molino.ratios.items():
            necesidad = molino.alimentacion_fresca * ratio / 100
            print(f"  {material.capitalize()}: {necesidad:.2f}t/h")

        recomendaciones = []
        for material, tolva in molino.tolvas.items():
            if material in molino.ratios:
                espacio_disponible = tolva.espacio_disponible()
                tiempo_llenado = espacio_disponible / (molino.alimentacion_fresca * molino.ratios[material] / 100)
                recomendacion = self.generar_recomendacion_especifica(molino, material, espacio_disponible)
                recomendaciones.append(recomendacion)
                print(f"Tiempo estimado para llenar {material.capitalize()}: {tiempo_llenado:.2f} horas")
            else:
                print(f"No se requiere {material.capitalize()} para el producto actual en {molino.nombre}")

        print("Restricciones:")
        print("Es necesario vaciar el sistema antes de una nueva alimentación para evitar contaminación.")

        return recomendaciones

    def generar_recomendacion_especifica(self, molino: Molino, material: str, espacio_disponible: float) -> str:
        if molino.nombre == "MC1":
            if material == "puzolana":
                return f"Alimentar Puzolana Húmeda a MC1 por MC1 (Espacio disponible: {espacio_disponible:.2f}t)"
            elif material == "yeso":
                if molino.tolvas["yeso"].nivel_actual + 2 <= molino.tolvas["yeso"].altura_max:
                    return f"Alimentar Yeso a MC1 por MC1 (Espacio disponible: {espacio_disponible:.2f}t)"
                else:
                    return f"Alimentar Yeso a MC1 por MC2 (Espacio disponible: {espacio_disponible:.2f}t)"
        elif molino.nombre == "MC2":
            if material == "puzolana_humeda":
                return f"Alimentar Puzolana Húmeda a 426HO04 por MC2 (Espacio disponible: {espacio_disponible:.2f}t)"
            elif material == "yeso":
                return f"Alimentar Yeso a MC2 por MC2 (Espacio disponible: {espacio_disponible:.2f}t)"
        elif molino.nombre == "MC3":
            if material == "puzolana":
                return f"Alimentar Puzolana Seca a MC3 por MC2 (Espacio disponible: {espacio_disponible:.2f}t)"
            elif material == "yeso":
                if self.mc1.tolvas["yeso"].nivel_actual > self.mc2.tolvas["yeso"].nivel_actual:
                    return f"Alimentar Yeso a MC3 por MC1 (Espacio disponible: {espacio_disponible:.2f}t)"
                else:
                    return f"Alimentar Yeso a MC3 por MC2 (Espacio disponible: {espacio_disponible:.2f}t)"
        
        return f"Alimentar {material.capitalize()} a {molino.nombre} (Espacio disponible: {espacio_disponible:.2f}t)"

    def aplicar_recomendaciones(self, recomendaciones: List[str]):
        for recomendacion in recomendaciones:
            partes = recomendacion.split()
            material = partes[1].lower()
            molino_destino = partes[3]
            cantidad = float(partes[-1].strip("t)"))

            if molino_destino == "MC1":
                self.mc1.tolvas[material].alimentar(cantidad)
            elif molino_destino == "MC2" or molino_destino == "426HO04":
                self.mc2.tolvas[material].alimentar(cantidad)
            elif molino_destino == "MC3":
                self.mc3.tolvas[material].alimentar(cantidad)

    def imprimir_niveles_tolvas(self):
        print("\nNiveles actuales de las tolvas:")
        for molino in [self.mc1, self.mc2, self.mc3]:
            print(f"\n{molino.nombre}:")
            for nombre_tolva, tolva in molino.tolvas.items():
                print(f"  {nombre_tolva}: {tolva.nivel_actual:.2f}m / {tolva.altura_max:.2f}m")
        print(f"\nSilo Blanco: {self.silo_blanco.nivel_actual:.2f}m / {self.silo_blanco.altura_max:.2f}m")
        print(f"Silo B: {self.silo_b.nivel_actual:.2f}m / {self.silo_b.altura_max:.2f}m")
        print(f"Galpón: {self.galpon.nivel_actual:.2f}m / {self.galpon.altura_max:.2f}m")

    def ajustar_velocidad_transportador(self, material: str):
        if material == "puzolana":
            return "Ajustar velocidad del transportador a 1100-1150 RPM"
        elif material == "yeso":
            return "Ajustar velocidad del transportador a 850-900 RPM"
        
    def verificar_contaminacion(self):
        return "ADVERTENCIA: Posible contaminación al vaciar el sistema. Proceder con precaución."

def main():
    sistema = SistemaAlimentacion()
    
    while True:
        sistema.set_productos()  # Preguntar por el tipo de producto para cada molino en cada ciclo
        
        recomendaciones = sistema.optimizar_alimentacion()
        
        print("\nRecomendaciones de alimentación:")
        for rec in recomendaciones:
            print(f"- {rec}")
            if "puzolana" in rec.lower():
                print(f"  {sistema.ajustar_velocidad_transportador('puzolana')}")
            elif "yeso" in rec.lower():
                print(f"  {sistema.ajustar_velocidad_transportador('yeso')}")
            print(f"  {sistema.verificar_contaminacion()}")
            print("  Esperar 3 minutos para vaciado de bandas")
        
        sistema.aplicar_recomendaciones(recomendaciones)
        
        print("\nNiveles actualizados de las tolvas:")
        sistema.imprimir_niveles_tolvas()
        
        continuar = input("\n¿Desea realizar otra optimización? (s/n): ").lower()
        if continuar != 's':
            break

if __name__ == "__main__":
    main()


Tipos de producto disponibles para MC1: P30, P40

Tipos de producto disponibles para MC2: P10, P16, P20, P30

Tipos de producto disponibles para MC3: P30

Optimizando MC1
Producto actual: P30
Necesidades:
  Clinker: 49.32t/h
  Puzolana: 21.60t/h
  Yeso: 1.08t/h
Tiempo estimado para llenar Clinker: 10.14 horas
Tiempo estimado para llenar Puzolana: 13.89 horas
Tiempo estimado para llenar Yeso: 277.78 horas
Restricciones:
Es necesario vaciar el sistema antes de una nueva alimentación para evitar contaminación.

Optimizando MC2
Producto actual: P10
Necesidades:
  Clinker: 60.90t/h
  Puzolana_humeda: 7.00t/h
  Yeso: 2.10t/h
Tiempo estimado para llenar Clinker: 4.93 horas
Tiempo estimado para llenar Puzolana_humeda: 71.43 horas
No se requiere Puzolana_seca para el producto actual en MC2
Tiempo estimado para llenar Yeso: 57.14 horas
Restricciones:
Es necesario vaciar el sistema antes de una nueva alimentación para evitar contaminación.

Optimizando MC3
Producto actual: P30
Necesidades:
  Cli